In [1]:
from src.util import seed_everything, update_avg, cleanup
from src.config import SEED, INTERIM_DATA_DIR

In [2]:
import os

if not INTERIM_DATA_DIR.exists():
    os.chdir("..")

In [3]:
seed_everything(SEED)

2023-11-06 00:39:45.547349: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-06 00:39:46.151444: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
import pandas as pd

from src.data.dataset import create_loaders

In [5]:
BATCH_SIZE = 128

In [6]:
train_loader, val_loader, test_loader = create_loaders(pd.read_csv(INTERIM_DATA_DIR / "swapped.csv"), batch_size=BATCH_SIZE)

In [7]:
from src.models.identity import IdentityModel
from src.metrics import calculate_all 

In [8]:
model = IdentityModel()

In [9]:
from tqdm import tqdm

avg_non_tox = 0
avg_bert = 0
avg_bleu = 0
for i, batch in (pbar := tqdm(enumerate(test_loader), total=len(test_loader))):
    inputs, target = batch
    train_pred = model.train(inputs, target)
    non_tox, bert, bleu = calculate_all(inputs, train_pred, target)
    avg_non_tox = update_avg(avg_non_tox, non_tox, i)
    avg_bert = update_avg(avg_bert, bert, i)
    avg_bleu = update_avg(avg_bleu, bleu, i)
    pbar.set_description(f"{avg_non_tox=:.3f} {avg_bert=:.3f} {avg_bleu=:.3f}")

  0%|          | 0/452 [00:00<?, ?it/s]

Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier_v1 were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
avg_non_tox=0.131 avg_bert=1.000 avg_bleu=0.173:  13%|█▎        | 58/452 [04:10<28:18,  4.31s/it] 


KeyboardInterrupt: 

In [ ]:
cleanup()